In [35]:
import pandas as pd

,gvkey,LPERMCO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,costat,sic
0,1000,23369,1970-12-31,1970.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
1,1000,23369,1971-12-31,1971.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
2,1000,23369,1972-12-31,1972.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
3,1000,23369,1973-12-31,1973.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
4,1000,23369,1974-12-31,1974.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321052,349994,59438,2023-10-31,2023.0,INDL,C,D,STD,CMND,CLEARMIND MEDICINE INC,USD,A,2834
321053,350681,58855,2021-12-31,2021.0,INDL,C,D,STD,GET,GETNET ADQUIRENCIA E,USD,I,7374
321054,351038,55612,2021-12-31,2021.0,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,A,2834
321055,351038,55612,2022-12-31,2022.0,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,A,2834


In [32]:
ticks = pd.read_csv ('../data/manual/ticks.csv', sep="|")
ticks['gvkey'] = ticks['gvkey'].fillna(0.0).astype(int)
ticks['Permco'] = ticks['Permco'].fillna(0.0).astype(int)
ticks

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647
...,...,...,...,...,...,...,...
106,"WERTHEIM SCHRODER & CO., INC.",Schroder Wertheim & Co. Inc.,NaN,6/24/1988,11/8/1990,0,0
107,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362
108,"WHITE, WELD & CO INC.","Merrill Lynch & Co., Inc.",NaN,2/26/1976,4/18/1978,21190,7267
109,"YAMAICHI INT'L (AMERICA), INC.",Yamaichi Securities Company Limited,NaN,9/29/1988,12/4/1997,0,0


In [3]:
import pandas as pd
import wrds
import config
from datetime import datetime

db = wrds.Connection(wrds_username=config.WRDS_USERNAME)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
# Function to convert date to quarter format 'YYYYQ#'
def date_to_quarter(date):
    year = date.year
    quarter = (date.month - 1) // 3 + 1
    return f"{year}Q{quarter}"

# Function to convert quarter 'YYYYQ#' to date format
def quarter_to_date(quarter):
    year = int(quarter[:4])
    quarter = int(quarter[-1])
    month = quarter * 3 
    return datetime(year, month, 1) + pd.DateOffset(months=1) - pd.DateOffset(days=1)


In [5]:
def fetch_financial_data_quarterly(pgvkey, start_date, end_date):
    """
    Fetch financial data for a given ticker and date range from the CCM database in WRDS.
    
    :param gvkey: The gvkey symbol for the company.
    :param start_date: The start date for the data in YYYY-MM-DD format.
    :param end_date: The end date for the data in YYYY-MM-DD format or 'Current'.
    :return: A DataFrame containing the financial data.
    """
    if not gvkey:  # Skip if no ticker is available
        return pd.DataFrame()
    
    # Convert 'Current' to today's date if necessary
    if end_date == 'Current':
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Convert start and end dates to datetime objects
    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)
    
    # Format start and end quarters
    start_qtr = date_to_quarter(start_date_dt)
    end_qtr = date_to_quarter(end_date_dt)
    
    query = f"""
    SELECT datafqtr, atq AS total_assets, ltq AS book_debt, ceqq AS book_equity, cshoq*prccq AS market_equity, gvkey, conm
    FROM comp.fundq as cst
    WHERE cst.gvkey = '{str(pgvkey).zfill(6)}'
    AND cst.datafqtr BETWEEN '{start_qtr}' AND '{end_qtr}'
    AND indfmt='INDL'
    AND datafmt='STD'
    AND popsrc='D'
    AND consol='C'
    """
    data = db.raw_sql(query)
    return data

Get all data for primary dealers

In [6]:
empty_tickers = []
prim_dealers = pd.DataFrame()

# Iterate over DataFrame rows and fetch data for each ticker
for index, row in ticks.iterrows():
    gvkey = row['gvkey']
    start_date = row['Start Date']
    end_date = row['End Date']     # Formatting date for the query
    
    # Fetch financial data for the ticker if available
    new_data = fetch_financial_data_quarterly(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['Ticker']:gvkey})
    else:
        prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
print(empty_tickers)

/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/2578673110.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/2578673110.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/2578673110.py:15: FutureWarning: The behavior of DataFrame 

[]


Get data from all firms in Compustat

In [7]:
prim_dealers = prim_dealers.drop_duplicates()
prim_dealers['datafqtr'] = prim_dealers['datafqtr'].apply(quarter_to_date)
prim_dealers

,datafqtr,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1993-09-30,53855.500,50158.000,3317.400,8450.487500,008007,WELLS FARGO & CO
0,2009-09-30,320258.165,297446.551,22666.180,27144.036200,015613,NOMURA HOLDINGS INC
0,2000-09-30,235646.000,224070.000,9904.000,16728.382500,015580,BANK OF MONTREAL
0,1999-03-31,NaN,NaN,NaN,NaN,015613,NOMURA HOLDINGS INC
0,1973-09-30,3243.754,3046.935,171.149,182.499905,007982,NORTHERN TRUST CORP
...,...,...,...,...,...,...,...
243,2022-12-31,3665743.000,3373411.000,264928.000,393483.997800,002968,JPMORGAN CHASE & CO
244,2023-03-31,3744305.000,3441223.000,275678.000,380803.479590,002968,JPMORGAN CHASE & CO
245,2023-06-30,3868240.000,3555724.000,285112.000,422661.002400,002968,JPMORGAN CHASE & CO
246,2023-09-30,3898333.000,3580962.000,289967.000,419253.980160,002968,JPMORGAN CHASE & CO


In [9]:
quarterly_sum_pd = prim_dealers.groupby('datafqtr').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum'
}).reset_index()
quarterly_sum_pd

,datafqtr,total_assets,book_debt,book_equity,market_equity
0,1962-03-31,8.469438e+03,4.396883e+03,766.714,1.943003e+03
1,1962-06-30,1.234590e+04,4.453320e+03,1124.091,1.344801e+03
2,1962-09-30,8.315135e+03,4.285148e+03,785.754,1.551048e+03
3,1962-12-31,1.339509e+04,4.784383e+03,1162.263,1.683967e+03
4,1963-03-31,9.003823e+03,4.611020e+03,801.393,1.842457e+03
...,...,...,...,...,...
243,2022-12-31,2.831133e+07,2.639202e+07,1697462.587,1.809920e+06
244,2023-03-31,2.873986e+07,2.677613e+07,1763940.663,1.810616e+06
245,2023-06-30,2.900576e+07,2.705667e+07,1731664.159,1.833932e+06
246,2023-09-30,2.907256e+07,2.712041e+07,1749305.177,1.818464e+06


In [10]:
def pull_CRSP_Value_Weighted_Index():
    """
    Pulls a value-weighted stock index from the CRSP database.

    Returns:
    - pandas.DataFrame: DataFrame containing the value-weighted stock index data.

    Note:
    This function executes a SQL query to retrieve the value-weighted stock index data from CRSP. 
    The returned DataFrame includes columns for 'date' and 'vwretd' (value-weighted return including dividends).
    """
    
    sql_query = """
        SELECT date, vwretd
        FROM crsp.msi as msi
        WHERE msi.date >= '1970-01-01' AND msi.date <= '2012-12-31'
        """
    
    data = db.raw_sql(sql_query, date_cols=["date"])
    return data

In [11]:
data = pull_CRSP_Value_Weighted_Index()

In [12]:
data

,date,vwretd
0,1970-01-30,-0.073254
1,1970-02-27,0.056706
2,1970-03-31,-0.004729
3,1970-04-30,-0.105318
4,1970-05-29,-0.064346
...,...,...
511,2012-08-31,0.026317
512,2012-09-28,0.026513
513,2012-10-31,-0.014055
514,2012-11-30,0.006217


In [41]:
linktable = pd.read_csv ('../data/manual/updated_linktable.csv', sep=",")
linktable = linktable.rename(columns={'GVKEY': 'gvkey'})
linktable

,gvkey,LPERMCO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,costat,sic
0,1000,23369,1970-12-31,1970.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
1,1000,23369,1971-12-31,1971.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
2,1000,23369,1972-12-31,1972.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
3,1000,23369,1973-12-31,1973.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
4,1000,23369,1974-12-31,1974.0,INDL,C,D,STD,AE.2,A & E PLASTIK PAK INC,USD,I,3089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321052,349994,59438,2023-10-31,2023.0,INDL,C,D,STD,CMND,CLEARMIND MEDICINE INC,USD,A,2834
321053,350681,58855,2021-12-31,2021.0,INDL,C,D,STD,GET,GETNET ADQUIRENCIA E,USD,I,7374
321054,351038,55612,2021-12-31,2021.0,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,A,2834
321055,351038,55612,2022-12-31,2022.0,INDL,C,D,STD,QNRX,QUOIN PHARMACEUTICALS LTD,USD,A,2834


In [42]:
merged = pd.merge(ticks, linktable, left_on='gvkey', right_on='gvkey')
merged = merged.drop_duplicates(subset=merged.columns.difference(['datadate','fyear']))
merged

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey,LPERMCO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,costat,sic
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504,31989,1997-12-31,1997.0,INDL,C,D,STD,ABNYY,ABN-AMRO HOLDINGS NV,USD,I,6020
11,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243,20483,1986-12-31,1986.0,INDL,C,D,STD,C,CITIGROUP INC,USD,A,6199
48,CITIGROUP GLOBAL MARKETS INC.,Citigroup Inc.,C (NYSE),6/15/1961,Current,20483,3243,20483,1986-12-31,1986.0,INDL,C,D,STD,C,CITIGROUP INC,USD,A,6199
85,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647,3151,1972-12-31,1972.0,INDL,C,D,STD,BAC,BANK OF AMERICA CORP,USD,A,6020
137,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647,3151,1972-12-31,1972.0,INDL,C,D,STD,BAC,BANK OF AMERICA CORP,USD,A,6020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,"SMITH BARNEY, HARRIS UPHAM & CO.,INC",Primerica Corporation,PRI,8/22/1979,8/31/1998,22177,1414,53383,2010-12-31,2010.0,INDL,C,D,STD,PRI,PRIMERICA INC,USD,A,6311
2386,SECURITY PACIFIC NATIONAL BANK,Security Pacific Corporation,6020,12/11/1986,1/17/1991,4205,9577,4205,1972-12-31,1972.0,INDL,C,D,STD,7448B,SECURITY PACIFIC CORP,USD,I,6020
2406,SOUTHERN CALIF S&L ASSOC,California Federal Savings and Loan Association,NaN,6/7/1983,8/5/1983,5884,2612,5884,1983-12-31,1983.0,INDL,C,D,STD,CAL.2,CALIFORNIA FED BANCORP INC,USD,I,6035
2419,TD SECURITIES (USA) LLC,The Toronto-Dominion Bank,"TD (NYSE, TSX)",2/11/2014,Current,29152,15706,29152,1996-10-31,1996.0,INDL,C,D,STD,TD,TORONTO DOMINION BANK,CAD,A,6020


In [45]:
linked_bd = linktable.drop_duplicates(subset=linktable.columns.difference(['datadate','fyear']))
linked_bd = linked_bd[((linked_bd['sic'] == 6211) | (linked_bd['sic'] == 6221)) & (~linked_bd['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_bd

,gvkey,LPERMCO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,costat,sic
1927,1148,420,1981-09-30,1981.0,INDL,C,D,STD,ADV.2,ADVEST GROUP INC,USD,I,6211
12885,1967,25401,1973-01-31,1972.0,INDL,C,D,STD,BAC.,BACHE GROUP INC,USD,I,6211
18487,2330,22881,1975-12-31,1975.0,INDL,C,D,STD,BSTNB,BOSTON CO INC -CL B,USD,I,6211
19743,2432,6441,1983-09-30,1983.0,INDL,C,D,STD,RCBI,BROWN (ROBERT C.) & CO INC,USD,I,6211
22074,2594,724,1974-08-31,1974.0,INDL,C,D,STD,CABO.1,CABOT (J.P.) EQUITY CORP,USD,I,6211
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309779,178073,55369,2015-12-31,2015.0,INDL,C,D,STD,RILY,B. RILEY FINANCIAL INC,USD,A,6211
310182,178519,53576,2010-12-31,2010.0,INDL,C,D,STD,LPLA,LPL FINANCIAL HOLDINGS INC,USD,A,6211
310684,178848,52781,2007-12-31,2007.0,INDL,C,D,STD,MKTSQ,DIRECT MARKETS HOLDINGS CORP,USD,I,6211
314909,185518,53607,2010-12-31,2010.0,INDL,C,D,STD,GLBR,GLOBAL BROKERAGE INC,USD,A,6211


In [46]:
empty_tickers = []
broker_dealers = pd.DataFrame()
for index, row in linked_bd.iterrows():
    gvkey = row['gvkey']
    start_date = '1970-01-01'
    end_date = '2012-12-31'    
    new_data = fetch_financial_data_quarterly(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['conm']:gvkey})
    else:
        broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
print(empty_tickers)

/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/3431501281.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/3431501281.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
/var/folders/hk/t7fnwnf568g5fqkjmc9d2vbr0000gn/T/ipykernel_34468/3431501281.py:11: FutureWarning: The behavior of Da

[]


In [51]:
broker_dealers = broker_dealers.drop_duplicates().sort_values(by=['datafqtr'])
broker_dealers

,datafqtr,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1970Q1,NaN,NaN,NaN,682.820179,008537,CITIGROUP GLOBAL MKTS HLDGS
0,1970Q1,NaN,NaN,NaN,NaN,003859,DELTEC INTL LTD -ADS
0,1970Q1,NaN,NaN,NaN,NaN,001967,BACHE GROUP INC
0,1970Q1,NaN,NaN,NaN,NaN,004037,DONALDSON LUFKIN & JENRETTE
1,1970Q2,NaN,NaN,NaN,NaN,003859,DELTEC INTL LTD -ADS
...,...,...,...,...,...,...,...
95,2012Q4,13.340,0.521,12.819,4.613600,021499,PAULSON CAPITAL CORP
3,2012Q4,20.257,2.382,17.875,NaN,020693,NORTHSTAR ASSET MGMT GRP INC
23,2012Q4,80.583,84.758,-5.103,9.300930,178073,B. RILEY FINANCIAL INC
75,2012Q4,14.455,12.557,1.898,2.144714,031159,GILMAN CIOCIA INC


In [66]:
quarterly_sum_bd = broker_dealers.groupby('datafqtr').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum'
}).reset_index()
quarterly_sum_bd = quarterly_sum_bd.rename(columns={'total_assets':'total_assets_bd', 'book_debt':'book_debt_bd', 'book_equity':'book_equity_bd','market_equity':'market_equity_bd' })
quarterly_sum_bd

,datafqtr,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
0,1970Q1,0.000,0.000,0.000,682.820179
1,1970Q2,0.000,0.000,0.000,453.086206
2,1970Q3,0.000,0.000,0.000,574.334575
3,1970Q4,0.000,0.000,394.533,788.528662
4,1971Q1,0.000,0.000,347.490,935.606373
...,...,...,...,...,...
167,2011Q4,318458.876,278103.137,33125.831,37481.822280
168,2012Q1,334002.562,294485.671,31607.289,43899.492233
169,2012Q2,332183.328,292158.035,33554.394,43024.142147
170,2012Q3,334579.162,293804.608,33726.748,41111.687732
